In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *
from CutFunctions import *
from SystematicsHelpers import *

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec

import warnings
warnings.filterwarnings("ignore")

import pyhf
from pyhf.contrib.viz import brazil

pyhf.set_backend("numpy")

In [ ]:
savePath = "../plot_files/06April2024_m200_v3_sensitivity/"

<h1> Read in HNL Dictionary </h1>

In [ ]:
nu_dict = np.load("../pkl_files/v3_April2024/nu_v3_dict.npy",allow_pickle='TRUE').item()

In [ ]:
cos_dict = np.load("../pkl_files/v3_April2024/cos_v3_dict.npy",allow_pickle='TRUE').item()

In [ ]:
bkg_dict = np.load("../pkl_files/v3_April2024/bkg_v3_dict.npy",allow_pickle='TRUE').item()

In [ ]:
hnl_dict = {}
mass_list = [200]

for m in mass_list:
    hnl_dict[m] = np.load("../pkl_files/v3_April2024/hnl_m"+str(m)+"_v3_dict.npy",allow_pickle='TRUE').item()

<h1>Sanity Plot</h1>

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12,4))

#-----------------------------------------------------------------#
ax1.step(bins, np.insert(hnl_dict[200]['signal'], 0, 0)
         , color = hnl_col
         , label =  "Signal"
)

#-----------------------------------------------------------------#
ax2.step(bins, np.insert(bkg_dict['bkg'], 0, 0)
         , color = nu_col
         , label =  "Neutrino + Cosmic Background"
)
#-----------------------------------------------------------------#
ax1.legend(loc = 'upper left',fontsize = 14)

plot_tick(ax1, 16)
plot_title(ax1, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  r"Slices (1$\times10^{21}$ POT)", 16)

ax1.set_xlim(xmin, xmax)
ax1.set_ylim(0, 80)
#-----------------------------------------------------------------#
ax2.legend(loc = 'upper left',fontsize = 14)

plot_tick(ax2, 16)
plot_title(ax2, "", 'Opt0 Time Corrected Z % 18.936 [ns]',  r"Slices (1$\times10^{21}$ POT)", 16)

ax2.set_xlim(xmin, xmax)
ax2.set_ylim(0, 2200)

#-----------------------------------------------------------------#
fig.tight_layout()

plt.show()

<h1>Define Models</h1>

In [ ]:
def make_model_stat_cosmics(m):
    model = pyhf.Model(
        {
      "channels": [
        {
          "name": "singlechannel",
          "samples": [
            {
              "name": "signal",
              "data": hnl_dict[m]['signal'],
              "modifiers": [
                {"name": "mu", "type": "normfactor", "data": None}, #This is the scaling which is to be calculated
                {"name": "signal_stat", "type": "staterror", "data": hnl_dict[m]['stat_err'] },
                {"name": "signal_cosmic", "type": "shapesys", "data": hnl_dict[m]['mistagging_err'] },
              ]
            },
            {
              "name": "background",
              "data": bkg_dict['bkg'],
              "modifiers": [
                {"name": "bkg_stat", "type": "staterror", "data": bkg_dict['stat_err']},
              ]
            }
          ]
        }
      ]
    }
    )

    print(f'Samples:\n {model.config.samples}')
    print(f'Modifiers are:\n {model.config.modifiers}')

    data = bkg_dict['bkg'] + model.config.auxdata
    
    return model, data

<h1>Stat + Cosmics</h1>

In [ ]:
model, data = make_model_stat_cosmics(200)

poi_vals = np.linspace(0, 1, 100)

obs_limit_single, exp_limits_single, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(data, 
                                                                                        model, 
                                                                                        poi_vals, 
                                                                                        level=0.1, 
                                                                                        return_results=True,
                                                                                        test_stat='qtilde')

print(f"Upper limit (obs): μ = {obs_limit_single:.4f}")
print(f"Upper limit (exp): μ = {exp_limits_single[2]:.4f}" + "\n")

EXP_LIMIT = np.sqrt(exp_limits_single[2]) * hnl_dict[200]['U']
LIMIT = np.sqrt(obs_limit_single) * hnl_dict[200]['U']

print(f"Expected limit is " + str(EXP_LIMIT))
print(f"Observed limit is " + str(LIMIT)+ "\n")

fig, ax = plt.subplots()
fig.set_size_inches(7, 5)
brazil.plot_results(poi_vals, results, ax=ax, test_size=0.10,)
plt.ylim(0.0,0.2)
fig.show()

<h1> Toys Time </h1>

In [ ]:
# mu' = 0: background-like
pars_bkg = model.config.suggested_init()
pars_bkg[model.config.poi_index] = 0.0
print(f"Background parameters: {list(zip(model.config.parameters, pars_bkg))}")

# mu' = 1: signal-like
pars_sig = model.config.suggested_init()
pars_sig[model.config.poi_index] = 1.0
print(f"Signal parameters: {list(zip(model.config.parameters, pars_sig))}")

# make the pdfs
pdf_bkg = model.make_pdf(pyhf.tensorlib.astensor(pars_bkg))
pdf_sig = model.make_pdf(pyhf.tensorlib.astensor(pars_sig))

In [ ]:
toy_calculator_qtilde = pyhf.infer.utils.create_calculator(
    "toybased",
    model.expected_data(pars_sig),
    model,
    ntoys=100,
    test_stat="qtilde",
)
qtilde_sig, qtilde_bkg = toy_calculator_qtilde.distributions(1.0)

In [ ]:
toy_calculator_qtilde.teststatistic(exp_limits_single[2])

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize = (6,4))

bins = np.linspace(0, 30, 26)
ax1.hist(
    qtilde_sig.samples,
    bins=bins,
    histtype="step",
    density=True,
    label=r"$f(\tilde{q}_1|1)$ signal-like",
    linewidth=2,
)
ax1.hist(
    qtilde_bkg.samples,
    bins=bins,
    histtype="step",
    density=True,
    label=r"$f(\tilde{q}_1|0)$ background-like",
    linewidth=2,
)


#ax1.set_xticks(np.arange(0, 10))
ax1.set_xlabel(r"(b) $\tilde{q}_1$", fontsize=18)
ax1.set_ylabel(r"$f\,(\tilde{q}_1|\mu')$", fontsize=18)
ax1.set_title(r"Alternative test statistic $(\tilde{q}_1)$ distributions")
ax1.legend()

ax1.set_yscale('log')
#plt.setp(ax1, xlim=(0, 9), ylim=(1e-3, 2), yscale="log")
#fig.set_size_inches(14, 6)
fig.tight_layout()

plt.show()

<h1>Plot</h1>

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(10,6))

xlimmin = 140
xlimmax = 240

plt.grid(axis = 'both', color='gainsboro', linestyle = ":")
#-------------------------------------------------------------------
ax1.scatter(200, TrippledBkgStatLimit, color = 'red', linestyle = '-', label = r'Signal Stat + Bkg Stat * 3')

ax1.scatter(200, StatLimit, color = 'orange', linestyle = '-', label = r'Signal Stat + Bkg Stat')

ax1.scatter(200, AThirdBkgStatLimit, color = 'blue', linestyle = '-', label = r'Signal Stat + Bkg Stat / 3')

#ax1.scatter(200, AThirdSignalStatLimit, color = 'orchid', linestyle = '-', label = r'Signal Stat / 3  + Bkg Stat')


ax1.scatter(200, noStatLimit, color = 'green', linestyle = '-', label = r'Without Stats Errors')
#-------------------------------------------------------------------

#plt.xlim(180, 220)
plt.ylim(4e-8, 1e-7)
ax1.set_yscale('log')

ax1.legend(loc="upper right", fontsize =14)

plot_tick(ax1, 16)
plot_title(ax1, "", "HNL Mass [MeV]",  "|U$_{\mu4}$|$^2$", 16)

#-------------------------------------------------------------------
fig.tight_layout()
plt.savefig(savePath+str("sensitivity_test.png"), dpi=200)
plt.show()